# 맛집 유튜브 채널 찾기
결과 : data/youtube_channel.xlsx  

방법1 : 목록을 정리해 놓은 글을 찾아서 정리한다.  
방법2 : [PLAYBOARD](https://playboard.co/)에서 특정 검색어(부산 맛집)를 입력해 크롤링한다.   
방법3 : 유튜브에서 특정 검색어(맛집)를 입력해 크롤링한다.  

# 유튜브 채널에서 추천한 맛집 찾기
~~방법1 : 유튜브API : 동영상은 제공해주지만 동영상 제목은 제공해주지 않습니다.~~  
방법 : 유튜브 크롤링

In [ ]:
# 사용한 라이브러리
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
import urllib
import numpy as na
import re

In [ ]:
# 유튜브 채널 가져오기
youtube_channels = pd.read_excel("./data/youtube_channel.xlsx", sheet_name="Sheet1")
# '제공'열 삭제
youtube_channels.drop('제공', axis=1, inplace=True)
# 중복 데이터 삭제
youtube_channels.drop_duplicates(keep='first')
display(youtube_channels.head(5))

In [ ]:
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)

In [ ]:
# 해당 채널 인기동영상 페이지 요청
# 모든 채널의 url이 저렇게 되지 않습니다.
key = youtube_channels.loc[9, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "") + "/videos?view=0&sort=p&flow=grid"
driver.get(url)
time.sleep(2)

# 츄릅켠 맛집 정보가져오기

In [ ]:
# 츄릅켠 홈 채널 들어가기
driver.get("https://www.youtube.com/c/ekekekTV/videos?view=0&sort=p&flow=grid")
# 30개의 동영상 url 가져오기
temp = []
video_titles = driver.find_elements_by_css_selector("#video-title")
for rank, video_title in enumerate(video_titles, 1):
    temp.append([rank, video_title.text, video_title.get_attribute("href")])

In [ ]:
# 2차원 배열 -> 데이터프레임
df = pd.DataFrame(temp, 
                   columns=['인기도 순위', '동영상 제목', '동영상 링크'])
display(df)

In [ ]:
df["정보"] = na.NaN
df["업로드날짜"] = na.NaN
for i in range(df.shape[0]):
    print(df.loc[i, "동영상 링크"])
    driver.get(df.loc[i, "동영상 링크"])
    time.sleep(5)
    info = driver.find_element_by_css_selector("#description > yt-formatted-string > span:nth-child(1)")
    print(info.text)
    df.loc[i, "정보"] = info.text
    date = driver.find_element_by_css_selector("#info-strings > yt-formatted-string")
    print(date.text)
    df.loc[i, "업로드날짜"] = date.text
    
df["맛집"] = na.NaN
for row in range(df.shape[0]):
    df.loc[row, "맛집"] = re.search("^\[.*", df.loc[row, "정보"]).group()[1:-1]
    
# 채널이름 넣기
df["정보제공"] = "츄릅켠"

# 더들리 맛집 정보가져오기

In [ ]:
# 더들리 홈 채널 들어가기
driver.get("https://www.youtube.com/c/%EB%8D%94%EB%93%A4%EB%A6%AC/videos?view=0&sort=p&flow=grid")
# 30개의 동영상 url 가져오기
temp = []
video_titles = driver.find_elements_by_css_selector("#video-title")
for rank, video_title in enumerate(video_titles, 1):
    temp.append([rank, video_title.text, video_title.get_attribute("href")])

In [ ]:
# 2차원 배열 -> 데이터프레임
df2 = pd.DataFrame(temp, 
                   columns=['인기도 순위', '동영상 제목', '동영상 링크'])
display(df2)

In [ ]:
df2["정보"] = na.NaN
df2["업로드날짜"] = na.NaN
df2["맛집"] = na.NaN
for i in range(df.shape[0]):
    print(df2.loc[i, "동영상 링크"])
    driver.get(df2.loc[i, "동영상 링크"])
    time.sleep(5)
    info = driver.find_element_by_css_selector("#description > yt-formatted-string > span:nth-child(3)")
    # print(info.text)
    df2.loc[i, "정보"] = info.text
    date = driver.find_element_by_css_selector("#info-strings > yt-formatted-string")
    print(date.text)
    df2.loc[i, "업로드날짜"] = date.text
    restaurant = driver.find_element_by_css_selector("#container > yt-formatted-string > a:nth-child(1)")
    print(restaurant.text)
    df2.loc[i, "맛집"] = restaurant.text

In [ ]:
for row in range(df.shape[0]):
    df2.loc[row, "맛집"] = df2.loc[row, "맛집"].replace("#","")

In [ ]:
# 채널이름 넣기
df2["정보제공"] = "더들리"

In [ ]:
df2

# 각 채널에서 생성된 데이터프레임 합치기

In [ ]:
# 나중에 반복문으로 돌려주세요
youtube_restaurant = pd.DataFrame()
youtube_restaurant = pd.concat([youtube_restaurant, df])
youtube_restaurant = pd.concat([youtube_restaurant, df2])

In [ ]:
youtube_restaurant.to_csv("./data/youtube_restaurant.csv",index = False, header=True)
youtube_restaurant

# 실험

In [ ]:
# 행찾기
youtube_channels.loc[9, '크리에이터']

In [ ]:
# 스크롤내리기
driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)

In [ ]:
# 정규표현식
import re
text = "[부산 서면 티파니돈까스]\n차장 (10분에 700원)\n 승용차부제, 경차, 친환경차 50% 감면"
re.search("^\[.*", text).group()[1:-1]

# 하다가 버린 코드

In [ ]:
"""
버린이유 : 바로 인기동영상 페이지로 가서 동영상을 가져올려고 한다. 이 코드는 바로 가지 않는다.
"""
# 해당 채널 페이지 요청
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "")
driver.get(url)
time.sleep(2)
# 동영상 페이지로 이동
video_link = driver.find_element_by_css_selector("#tabsContent > tp-yt-paper-tab:nth-child(4) > div")
video_link.click()
# 정렬기준 선택
video_link = driver.find_element_by_css_selector("#icon-label")
video_link.click()

In [ ]:
"""
버린이유 : flex 기술로 인해 드라이버가 모르는 태그는 작동을 시킬 수 없다.
"""
# 정렬기준 선택
video_link = driver.find_element_by_css_selector("#menu > a.yt-simple-endpoint.style-scope.yt-dropdown-menu.iron-selected > tp-yt-paper-item")
video_link.click()
time.sleep(1)

In [ ]:
"""
사용한 이유 : &기호를 잘못 변환해서 사용했다.
버린이유 :  왜 인지 인기동영상으로 넘어가지 않는다.
"""
# 해당 채널 페이지 요청
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/c/" + urllib.parse.quote(key).replace("%20", "") + "/videos?view=0&sort=p&flow=grid"
driver.get(url)
time.sleep(2)

In [ ]:
"""
버린이유 : 한번에 채널을 불러오는 것이 더 좋을 것이라고 판단해서
"""
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
# 키워드를 포함한 페이지 요청 
key = youtube_channels.loc[4, '크리에이터']
url = "https://www.youtube.com/results?search_query=" + urllib.parse.quote(key)
driver.get(url)
time.sleep(2)

In [ ]:
"""
버린이유 : 검색창에 글을 못 써서
"""
# 크롤링
# 드라이버 준비
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
# 유튜브 들어가기
driver.get("https://www.youtube.com/")
time.sleep(2)
# 검색어 입력
inputId = driver.find_element_by_id('search')
inputId.send_keys("맛상무")
inputId.send_keys(Keys.ENTER)

In [ ]:
"""
제공 : https://gist.github.com/WoongheeLee/70cb07f2fc1f32a76c82a99d84d51044
버린이유 : 서브 버튼 클릭이 아니라서
"""
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_image_title(url):
    # 웹 드라이버 초기화
    driver_path = 'C:/Temp/chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트

    idx = 1
    while True:
        try:
            img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            
            # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')
            
            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 8 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
            
            # 썸네일 주소를 리스트에 저장
            image = driver.find_element_by_xpath(img_xpath)
            img_url = image.get_attribute('src')
            image_list.append(img_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            print(idx,title.text,img_url)
            title_list.append(title.text)

            idx += 1
        except Exception as e:
            print()
            print(e)
            break
    assert len(image_list) == len(title_list)
    driver.close()
    return image_list, title_list

# 자이언트 펭TV
url1 = 'https://m.youtube.com/channel/UCtckgmUcpzqGnzcs7xEqMzQ/videos'
image1, title1 = get_image_title(url1)
print(title1)

# 이슈


해결방안 : https://www.python2.net/questions-895942.htm